# Finite elements in 1D

We reconsider the boundary value problem (BVP) from the  [finite difference notebook](FiniteDifferences1D.ipynb),
$$
-u''(x) + u(x) = f(x), \qquad  0 < x < 1,
$$
with boundary conditions 
$$
u(0) = u(1) = 0
$$
at both ends. In this noteboook, we treat it with finite elements.

## Weak form

One way to obtain the so-called weak form of the BVP is by multiplying the equation by a smooth **test function** $v$ that has the boundary conditions $v(0)=v(1)=0$ and integrating by parts. Thus, $\int_0^1 (-u'' + u ) v  = \int_0^1 f v $
becomes 
$$
\int_0^1 u' v' + u v = \int_0^1 f v.
$$
The solution $u$ of the BVP must satisfy this equation for all such  $v$. The function space of all admissible $v$ in this calculation is an infinite-dimensional space,  a subspace of the well-known Sobolev space $H^1(0,1)$. More about that later. The above displayed equation is called the *weak form* of the BVP because you only need square integrability of one derivative of $u$ for it to make sense. The function $u$ in the weak form is called the **trial function**.


## Finite element method

The finite element method imposes the same equation as above on a (computable) finite-dimensional space.


What should be the above-mentioned finite-dimensional space? This varies from problem to problem. We shall have much more to say about this later. For now, we set a uniform mesh $\{ x_i = i h \}$ of grid spacing $h$ and set  
$$
V_h = \{ v: \; v \text{ is continuous and }
v|_{[x_i, x_{i+1}]} \text { is linear }\}.
$$
This space is called the **lowest order Lagrange finite element space**.
So as not to forget the boundary conditions, let us also name the following subspace,
$$\newcommand{\Vo}{\mathring{V}}
\Vo_h = \{ v \in V_h :\; v(0)=v(1)=0\}.
$$

A Finite Element Method (FEM) for the above BVP 
selects a $u_h$ from the space $\Vo_h$ by requiring that
$$
\int_0^1 u_h' v' + u_h v = \int_0^1 f v \quad\text{ for all }v \in \Vo_h.
$$
This $u_h$ is the FE approximation to the exact solution $u$.

## Implementation aspects of FEM

We use the NGSolve package for finite elements. It uses the mesh generation package Netgen.

In [ ]:
import ngsolve as ng
import netgen.meshing as ngm  # meshing module
from netgen.csg import Pnt    # Points (or mesh vertices)

NGSolve optimizes its user interface to two and three-dimensional meshes (as most of its users do not want to bother with 1D). In 2D and 3D, meshes can be automatically generated by the mesh generator `Netgen` (included with the NGSolve distribution). However, unlike the two and three-dimensional case, in one-dimension, we will have to "make" the one-dimensional mesh ourselves.

In [ ]:
def make1dmesh(N):
    """Subdivide [0, 1] into a mesh of N elements"""
    
    m = ngm.Mesh(dim=1)
    h = 1.0 / N
    pnums = []
    
    for i in range(0, N+1):
        
        # add points (use just one (first) coordinate)
        pnums.append(m.Add(ngm.MeshPoint(Pnt(i*h, 0, 0))))

    # name interior region (more useful when there are many)
    domain = m.AddRegion("mymaterial", dim=1)
    
    for i in range(0, N):

        # add 1D elements
        m.Add(ngm.Element1D([pnums[i], pnums[i+1]], index=domain))

    # name two boundary regions: left and right end points
    leftboundary = m.AddRegion("left", dim=0)
    rghtboundary = m.AddRegion("right", dim=0)

    m.Add(ngm.Element0D(pnums[0],  index=leftboundary)) 
    m.Add(ngm.Element0D(pnums[-1], index=rghtboundary))

    return m

The above makes a basic `Netgen` mesh object using `MeshPoint` and `Element1D` objects. Next, we convert it to an `NGSolve` mesh which can be passed as argument to NGSolve's finite element functionalities.

In [ ]:
N = 5   # the number of mesh elements
m = make1dmesh(N)
mesh = ng.Mesh(m)

A mesh object can be queried in many ways - type `help(mesh)` for the documentation. For example, you can double check it has the points you intended to set as the grid points:

In [ ]:
for x in mesh.vertices:
    print(x.point)

## Lagrange finite element space in 1D

The next steps use functionalities that are common for any finite element space (be it in 1D, 2D, or 3D) implemented in NGSolve.   Here is the NGSolve representation of the space $V_h$ we introduced above.

In [ ]:
Vh = ng.H1(mesh, order=1)  # Lowest order is 1 (linear)

The syntax `H1` comes from the fact that Lagrange finite elements are used to approximate weak formulations in the Sobolev space $H^1$.

Functions in this space are represented in  NGSolve by  `GridFunction` objects. Let us see how a function in this space looks like.

In [ ]:
v = ng.GridFunction(Vh, 'myfun')

This is now an uninitialized function. We can set values of `v` in various ways. One way is to interpolate a known function.


#### Interpolation into the finite element space

Functions can be interpolated into finite element spaces. In NGsolve this is accomplished using the `Set` method.  To declare functions in terms of coordinates (just $x$ in 1D, $x, y, z$ in 3D etc), these coordinates are available as NGSolve `CoefficientFunction` objects.

In [ ]:
from ngsolve import x     

v.Set(x * x)

To visualize `v`, we may use the common `matplotlib` python module as follows.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plotv(v=v, sty='.-'):
    pts = [vtx.point[0] for vtx in mesh.vertices]
    plt.plot(pts, np.array(v.vec), sty)
    plt.xlabel('x'); plt.ylabel('$v$');

plotv()

As you can see, we now have a piecewise linear continuous approximation of $x^2$, i.e., an approximation of $x^2$ from the FE space $V_h$.

## Basis expansion

The functions $\psi_i \in V_h$ with the property 
$$
\psi_i(x_j) = \delta_{ij}
$$
at every mesh point $x_j$ are often called **hat functions**. Here $\delta_{ij}$ denotes the Kronecker delta (equals 1 if $i=j$ and 0 otherwise). Although the above only gives the values of $\psi_i$ at the mesh vertices $x_j$, that is enough to determine the function $\psi_i$ everywhere since it is linear in between the mesh points. Here is a visualization of one of these hat functions (obtained by setting them in `v`'s memory).

In [ ]:
i = N//2
v.vec[:] = 0
v.vec[i] = 1
plotv()

And here are all of them:

In [ ]:
for i in range(N+1):
    v.vec[:] = 0
    v.vec[i] = 1
    plotv()
plt.title('$\psi_i(x)$ for each $i$');

(You should prove that $\psi_i$ for all $i=0, \ldots, N$ forms a basis for the space $V_h$.)

In fact, when  `v` is expressed in term of finite element basis of hat functions $\{\psi_i\}$, 
$$
v(x) = \sum_{i=0}^N v_i \psi_i(x)
$$
the vector of coefficients $v_i$ in this basis expansion
gives all information contained in `v`. This is the vector we have been accessing using `v.vec`.

In [ ]:
v.Set(x * x)  # vector of coefficients in the basis expansion 
print(v.vec)  # of the Lagrange interpolant of x*x

## Making the FE system


To find $u_h\in V_h$ satisfying 
$$
\int_0^1 u_h' v' + u_h v = \int_0^1 f v \quad\text{ for all }v \in V_h.
$$
we employ the hat function basis $\psi_i$. Expanding 
$$
u_h(x) = \sum_i c_i \psi_i(x)
$$
we know that finding $u_h(x)$ is the same as finding the numbers $c_i$. Setting $v$ to $\psi_i$ above and using the basis expansion of $u_h$, we have 
$$
\sum_i \int_0^1 ( \psi_j' \psi_i' + \psi_j \psi_i) c_j = \int_0^1 f \psi_i.
$$
Thus solving for the FE solution is the same as solving the linear system
$$
A c = b 
$$
where $c = (c_i)$ contains the coefficients of $u_h$, and 
$$
A_{ij} = \int_0^1 \psi_j' \psi_i' + \psi_j \psi_i, 
\qquad 
b_i = \int_0^1 f \psi_i.
$$

All finite element codes will provide you facilities to make such linear systems. There is a particular way to do this, called **assembly,** which exploits the fact that the basis functions $\Psi_i$ within an element $[x_k, x_{k+1}]$, being polynomial, are particularly simple to integrate. Namely, we split
$$
A_{ij} = \sum_{k=0}^N \int_{x_k}^{x_{k+1}} 
\psi_j' \psi_i' + \psi_j \psi_i, 
\qquad 
b_i = \sum_{k=0}^N \int_{x_k}^{x_{k+1}}   f \psi_i.
$$
To make the matrix $A$, one computes the **local element matrices** for each element  $k$, 
$$
A_{ij}^{(k)} = \int_{x_k}^{x_{k+1}} 
\psi_j' \psi_i' + \psi_j \psi_i
$$
and sums up the element contributions (and similarly for the right hand side vector $b$).  Note that on element $[x_k, x_{k+1}]$ only the cases $i, j \in \{ k, k+1\}$ produce a nonzero $A_{ij}^{(k)}$.

Using NGSolve, you can compute local matrices (which in the current example happens to be very easy to compute even by hand) using a "symbolic bilinear form integrator" or `SymbolicBFI` as follows. The reason for the name is that we think of $\int u' v' + uv$ as an integral that is (bi)linear in $u$ and $v$. Note that the derivative computation in $u' v'$ is accomplished by `grad` below (which provides a unified syntax for gradient in all dimensions).

In [ ]:
from ngsolve import SymbolicBFI, grad

u = Vh.TrialFunction()  # symbolic trial function 
v = Vh.TestFunction()   # and test function objects
a = ng.SymbolicBFI(grad(u) * grad(v) + u * v)

NGSolve maintains the basis functions $\psi_i$ for each finite element space. So given the `SymbolicBFI`, it now has all the information needed to  calculate and display the local element matrix of any element.

In [ ]:
k = 2   # select element

els = list(Vh.Elements())
el = els[k]  

# calculate & print out the local element matrix
elmat = a.CalcElementMatrix(el.GetFE(), el.GetTrafo())
print(elmat)

(Does this match what you compute "by hand"?)

We can now of course continue to put together the element matrices to make the matrix $A$. In fact, NGSolve can do the assembly for you. But before that, let's not forget the boundary conditions.

## Essential boundary conditions 

Boundary conditions that are incorporated into a finite element space are called **essential** boundary conditions. Note that we need to find the FE solution in $\Vo_h$, a space which has essential boundary conditions. We can indicate that essential boundary conditions are required on certain parts of boundary by specifying those parts within the `dirichlet` keyword, a shown below:

In [ ]:
Vo = ng.H1(mesh, order=1, dirichlet='left|right') 

Here `left` and `right` are the same names we chose for the boundary regions when we constructed the mesh in `make1dmesh` and  `|` indicates their union.

In [ ]:
mesh.GetBoundaries()

When a boundary region is marked using the `dirichlet` keyword, 
the basis hat functions associated to that region is 
considered **constrained** by boundary conditions. Only the remaining basis functions are **free** degrees of freedom. (This is the rationale for the term `FreeDofs` that appears below.)

## Solve for FE solution

We consider the case $f = (1+\pi^2) \sin(\pi x)$ where we know  the exact solution should be $u = \sin(\pi x)$.

In [ ]:
from math import pi
from ngsolve import dx, sin

f = (1+pi*pi)*sin(pi*x)

u, v = Vo.TnT()   # TnT gets both TestFunction and TrialFunction

a = ng.BilinearForm(Vo)
a += grad(u) * grad(v) * dx  + u * v * dx

b = ng.LinearForm(Vo)
b += f * v * dx

a.Assemble()   # assemble the left hand side matrix
b.Assemble()   # assemble the right hand side vector

# invert the linear system, restricting to the free degrees of freedom
uh = ng.GridFunction(Vo, 'uh')
uh.vec.data = a.mat.Inverse(Vo.FreeDofs()) * b.vec

In [ ]:
plotv(uh)

On a finer mesh, you should see the plot of the solution approaching the exact solution $\sin(\pi x)$. You can repeat the above calculations for a finer mesh by simply changing the value of `N` set above. 

## Exercises


#### Exercise 1  

Consider the lowest order Lagrange finite element space $V_h$ built on a one-dimensional mesh $x_0 < x_1 < \cdots < x_N$ of the interval $[x_0, x_N]$. Let  $\psi_i \in V_h$ satisfy
$
\psi_i(x_j) = \delta_{ij}
$
at every mesh point $x_j$.
Prove that the $\{\psi_i:  i=0, \ldots, N\}$ is a basis for the space $V_h$.

$\newcommand{\ii}{\hat{\imath}} $

#### Exercise 2

Derive a weak formulation for the
  following boundary value problem, where $k>0$ is a real number
  (called the wavenumber) and $\ii$ is the imaginary unit.
 $$\begin{aligned}
    u'' + k^2 u & = 0, && \text{ in the interval } (0, 1),
    \\
    u(0) & = 1, \\
    u'(1)  - \ii k u(1) & = 0.
  \end{aligned}
  $$
  
  
#### Exercise 3 

Use Lagrange finite elements to approximately solve the boundary value problem using the weak formulation of Exercise 2. You will need  complex-valued Lagrange finite element functions (available in NGSolve using the `complex` keyword in `ng.H1(...)`). Your code should take $k$ and the  number of elements as input parameters. Calculate the exact solution   ``by hand'' and compare it with your numerical  solution. How do the numerical solutions change as you vary $k$?